<h1>Compute 95% Confidence Interval for SIR Predictions</h1>

<p>In this tutorial we are going how to insert uncertainties in our preditctions. For achive it we are going to compute a non-parametric interval around our predictions. The interval is created using a techinique called bootstrap.</p>

<p>It's easy to compute confidence interval using the class SIR_EDO. This class now have a method calling fit_predictCI that returns a pandas data frame, with all you need to put uncertainties around your predictions. This methods take advantage of multprocessing system and can be distributed across your system.</p>

In [1]:
import multiprocessing as mp
import sys
#add the path were the models are
sys.path.append("../main/")
import pandas as pd
import numpy as np
from plotnine import *
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from itertools import repeat
import time
from functools import reduce
from modelos import SIR_EDO
#set default theme for plts
theme_set(theme_linedraw())

In [2]:
data = pd.read_csv("../data/estados.csv")
#Select only Sao Paulo
sp = data[data["state"] == "SP"]
#Remove missing values to not crash the intervals
sp = sp.dropna()
#create a series with the cummulative number of cases
y = sp["TOTAL"]

#Give the number of days since the day of first case confirmed
x = range(1,len(sp["TOTAL"]) + 1)

In [3]:
#start model
model = SIR_EDO(50000000)

In [4]:
model.fit(x = x, y = y)

In [5]:
resultdf = model.predictCI(x =x, y = y, ndays = 30, start = "2020-02-25",  bootstrap = 100, n_jobs = 8)

In [6]:
resultdf

,date,Pred,Pred_lb,Pred_ub,date,I,I_lb,I_ub,date,R,R_lb,R_ub,date,S,S_lb,S_ub
0,2020-02-25,1.000000,1.000000,1.000000,2020-02-25,1.000000,1.000000,1.000000,2020-02-25,0.000000,0.000000,0.000000,2020-02-25,5.000000e+07,5.000000e+07,5.000000e+07
1,2020-02-26,1.382190,1.327878,1.416290,2020-02-26,1.174545,1.130689,1.205293,2020-02-26,0.207646,0.164655,0.219396,2020-02-26,5.000000e+07,5.000000e+07,5.000000e+07
2,2020-02-27,1.831162,1.707249,1.915483,2020-02-27,1.379806,1.278407,1.452504,2020-02-27,0.451356,0.359443,0.481670,2020-02-27,5.000000e+07,5.000000e+07,5.000000e+07
3,2020-02-28,2.358833,2.146106,2.512418,2020-02-28,1.621301,1.445395,1.750267,2020-02-28,0.737532,0.589929,0.794484,2020-02-28,5.000000e+07,5.000000e+07,5.000000e+07
4,2020-02-29,2.980848,2.632884,3.232566,2020-02-29,1.906274,1.634581,2.109995,2020-02-29,1.074574,0.856984,1.169003,2020-02-29,5.000000e+07,5.000000e+07,5.000000e+07
5,2020-03-01,3.732609,3.196586,4.125164,2020-03-01,2.251069,1.853667,2.555920,2020-03-01,1.481540,1.175120,1.630693,2020-03-01,5.000000e+07,5.000000e+07,5.000000e+07
6,2020-03-02,4.624346,3.850509,5.203831,2020-03-02,2.660408,2.104895,3.095016,2020-03-02,1.963939,1.553461,2.186408,2020-03-02,5.000000e+07,4.999999e+07,5.000000e+07
7,2020-03-03,5.656542,4.593186,6.487325,2020-03-03,3.134532,2.388268,3.729484,2020-03-03,2.522011,1.989791,2.836395,2020-03-03,4.999999e+07,4.999999e+07,5.000000e+07
8,2020-03-04,6.869404,5.416108,8.058182,2020-03-04,3.692624,2.703783,4.510069,2020-03-04,3.176780,2.492565,3.626075,2020-03-04,4.999999e+07,4.999999e+07,4.999999e+07
9,2020-03-05,8.324280,6.343519,9.958425,2020-03-05,4.362759,3.056781,5.457983,2020-03-05,3.961521,3.035031,4.590361,2020-03-05,4.999999e+07,4.999999e+07,4.999999e+07


In [7]:
#plot graph using ggplot
(ggplot(resultdf) + 
    geom_line(aes(x = "date", y = "Pred")) +
    geom_ribbon(aes(x = "date", ymin = "Pred_lb", ymax = "Pred_ub"), alpha = 0.5) +
    scale_x_datetime(breaks = date_breaks('5 days'), labels=date_format('%d-%b')) +
    ggtitle("Series of covid-19 for Sao Paulo") +
    ylab("Number of cases"))

ValueError: Wrong number of items passed 4, placement implies 1